In [4]:
import os 
import re
current_dir = os.getcwd()
parent_dir =  os.path.dirname(current_dir)
script_dir = os.path.join(parent_dir, 'RPE/train_simplified.py')

args_command = '--vocab-size 3 --seq-length 100 --n-heads 1 --n-gram 2 --w-plus 3 --c-alpha 0.01 --device cuda --n-epochs 50000   --lr 1 --train-cmd CWa --low-degree 1 --dataset Markov' # Markov dataset
# args_command = '--vocab-size 3 --seq-length 100 --n-heads 3 --n-gram 3 --w-plus 3 --c-alpha 0.01 --device cuda --n-epochs 50000   --lr 1 --train-cmd CWa --low-degree 2' # N-gram dataset

prefix = '''#!/bin/bash
#SBATCH --job-name=<YOUR_JOB_NAME> # Job name
#SBATCH --partition=gpu 
#SBATCH --gres=gpu:1 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=8
#SBATCH --time=24:00:00 
#SBATCH --output=<YOUR_OUTPUT_FILE>
#SBATCH --error=<YOUR_ERROR_FILE>
#SBATCH --requeue 

echo '-------------------------------'
cd ${SLURM_SUBMIT_DIR}
echo ${SLURM_SUBMIT_DIR}
echo Running on host $(hostname)
echo Time is $(date)
echo SLURM_NODES are $(echo ${SLURM_NODELIST})
echo '-------------------------------'
echo -e '\\n\\n'

export PROCS=${SLURM_CPUS_ON_NODE}

# Set the working directory
cd <YOUR_WORKING_DIR>

module load miniconda
conda activate scgpt1
'''

prefix = prefix.replace('<YOUR_OUTPUT_FILE>', os.path.join(current_dir, 'slurm_output/%j.out'))
prefix = prefix.replace('<YOUR_ERROR_FILE>', os.path.join(current_dir, 'slurm_output/%j.err'))
prefix = prefix.replace('<YOUR_JOB_NAME>', '2-gram_experiment')
prefix = prefix.replace('<YOUR_WORKING_DIR>', os.path.dirname(script_dir))
# Extract n-gram value from args_command
n_gram_match = re.search(r'--n-gram (\d+)', args_command)
n_gram_value = n_gram_match.group(1) if n_gram_match else 'unknown'

script_name = f'{n_gram_value}-gram.sh'
with open(script_name, 'w') as file:
    file.write(prefix)
    # add new line 
    file.write('\n')
    file.write(f'python3 {script_dir} {args_command}')
    file.write('\n')